In [1]:
#! pip install cherrypy

In [2]:
#! pip install markupsafe==2.0.1

In [3]:
#! pip install itsdangerous==2.0.1

In [4]:
#! pip install -U Werkzeug==0.16.0

In [5]:
import pandas as pd
import numpy as np
from app_function import view_classifica, view_player_page, view_role_explorer

In [6]:
team_list = []
global team_list
modulo_scelto = '3-4-3'
global modulo_scelto 

In [7]:
player_info = pd.read_csv('../dataset/stat_info^(Stagione 2021_22).csv',index_col=[0])
player_hist = pd.read_csv('../dataset/player_history^(Stagione 2021_22).csv',index_col=[0])

player_hist.Voto = player_hist.Voto.astype(float)
player_hist.FV = player_hist.FV.astype(float)
player_hist.Bonus = player_hist.Bonus.astype(float)
player_hist.Malus = player_hist.Malus.astype(float)

all_names = player_info.Nome.unique().tolist()
all_teams = player_info.Squadra.unique().tolist()

classifica = pd.read_csv('../dataset/final_classifica^(Stagione 2021_22).csv',index_col=[0])

In [8]:
def view_builded_team(team_dict):
    
    team_df = player_info.loc[player_info.Nome.isin(team_dict)]
    
    counts_role = team_df.Ruolo.value_counts()
    
    n_dif = counts_role.loc['D']
    n_cen = counts_role.loc['C']
    n_att = counts_role.loc['A']
    
    dif_class = 'dif_'+ str(n_dif)
    cen_class = 'cen_'+ str(n_cen)
    att_class = 'att_'+ str(n_att)
    
    role_total = team_df.groupby('Ruolo')[['Bonus','Malus','Gol','Assist','xG','xA']].sum()
    role_mean = team_df.groupby('Ruolo')[['media_voto','media_fantavoto','%_con_voto', '%_fv_5', '%_fv_5_6', '%_fv_6_7', '%_fv_7_8', '%_fv_>_8',]].mean()
    
 
    table = role_total.merge(role_mean, left_index=True,right_index=True).loc[['P','D','C','A']].reset_index()
    table = np.round(table,2)
    
    return render_template( 'builded_team.html', table = table)

In [9]:
def view_team_builder(modulo_scelto):
    
    grid_list = modulo_scelto.split('-')
    dif_class = 'dif_'+grid_list[0]
    cen_class = 'cen_'+grid_list[1]
    att_class = 'att_'+grid_list[2]

    n_dif = int(grid_list[0])
    n_cen = int(grid_list[1])
    n_att = int(grid_list[2])

    nomi_dif = ['nome_' + str(2+i) for i in range(0,n_dif)]
    nomi_cen = ['nome_' + str(n_dif+2+i) for i in range(0,n_cen)]
    nomi_att = ['nome_' + str(n_dif+n_cen+2+i) for i in range(0,n_att)]

    por_names = player_info.loc[player_info.Ruolo == 'P'].Nome.unique()
    dif_names = player_info.loc[player_info.Ruolo == 'D'].Nome.unique()
    cen_names = player_info.loc[player_info.Ruolo == 'C'].Nome.unique()
    att_names = player_info.loc[player_info.Ruolo == 'A'].Nome.unique()

    return render_template( 'team_builder.html',  modulo_scelto = modulo_scelto,
                           por_names = por_names, dif_names = dif_names, cen_names = cen_names, att_names = att_names, 
                           dif_class=dif_class, cen_class=cen_class, att_class=att_class,
                           n_dif = n_dif,n_cen = n_cen,n_att = n_att,
                           nomi_dif= nomi_dif,
                           nomi_cen = nomi_cen,
                           nomi_att = nomi_att
                          )

In [ ]:
from flask import Flask, render_template,request, redirect, url_for

app = Flask(__name__)

@app.route("/")
def home_page():
    return render_template('home_page.html')

@app.route("/player-page")
def player_page(nome=all_names[0]):
    return view_player_page(player_info,player_hist,all_names,nome)
    
@app.route('/player-page', methods=['POST'])
def handle_name():
    nome_selected= request.form.get('nome_selezionato')
    return view_player_page(player_info,player_hist,all_names,nome_selected)

@app.route('/classifica', methods=("POST", "GET"))
def classfica_page():
    return view_classifica(classifica)

@app.route('/role-explorer')
def role_page():
    return view_role_explorer(player_info, all_teams )

@app.route('/role-explorer', methods=['POST'])
def handle_filter():
    min_pres = request.form.get('min_pres')
    role_selected = request.form.get('role_selected')
    team_selected = request.form.get('team_selected')
    return view_role_explorer(player_info, all_teams,
                              role = role_selected, 
                              filter_presence = min_pres, 
                              team_selected = team_selected)

@app.route('/team-builder')
def team_builder():
    return view_team_builder(modulo_scelto)
            
@app.route('/team-builder', methods=['POST'])
def handle_team_builder():
    team_list = []
    if 'scegli_modulo' in request.form:
        modulo_scelto = request.form.get('modulo_scelto')
        return view_team_builder(modulo_scelto)
    
    if 'build_team' in request.form:
        for i in range(1,12):
            team_list.append(request.form.get('nome_'+str(i)))
        return view_builded_team(team_list)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2022 13:58:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 13:58:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Jul/2022 13:58:32] "GET /role-explorer HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 13:59:03] "POST /role-explorer HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 13:59:16] "POST /role-explorer HTTP/1.1" 200 -
